In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def split_data(input_file, train_set, test_set):

In [2]:

df = pd.read_json("./datasets/language_detection/merged_dataset.jsonl", lines=True)
df.head()



,text,label,format,source,citation
0,"Oro Yoruba so pe, Ore dani ko to pon, abinibi ...",yoruba,file,{'provider': 'Niger-Volta Language Technologie...,c2
1,"A mo pe iko to gbayi ni Naijiria, won ma maa l...",yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
2,Igba mesan-an ni egbe agbaboolu obinrin Naijir...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
3,Opolopo odo aye ode oni ti ko asa fi fe iyawo ...,yoruba,file,{'provider': 'Niger-Volta Language Technologie...,c2
4,"Ka ma gbagbe, isele kan wa to ti bere si ni gb...",yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None


In [3]:
df["text"].duplicated().sum()

540

0

In [13]:
duplicates = df[df["text"].duplicated() == True]
duplicates.head()

,text,label,format,source,citation
1079,This article contains content provided by Twit...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
3022,This article contains content provided by Twit...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
3277,This article contains content provided by Twit...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
5295,"Orisun aworan, Getty Images",yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None
6578,This article contains content provided by Twit...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None


In [20]:
duplicates["text"].iloc[0]

"This article contains content provided by Twitter. We ask for your permission before anything is loaded, as they may be using cookies and other technologies. You may want to read Twitter cookie policy and privacy policy before accepting. To view this content choose 'accept and continue'."

In [32]:
df[df["text"].str.contains("This article contains content provided by Twitter. We ask for your permission before anything is loaded", case = False)]

,text,label,format,source,citation
179,This article contains content provided by Twit...,yoruba,article,"{'provider': 'BBC', 'url': 'https://www.bbc.co...",None


In [4]:
df["label"].unique()

array(['yoruba', 'igbo', 'hausa', 'english', 'pidgin'], dtype=object)

In [7]:
eng_words = ["The", "This", "it", "We", "It", "She", "He", "is"]

# word boundaries so 'is' doesn't match inside words
pattern = r"\b(" + "|".join(eng_words) + r")\b"

# Change eq("language")
mask = df["label"].eq("hausa") & df["text"].str.contains(pattern, case=False, regex=True)

# Print matching entries
# print(len(df[mask])) 
# yoruba: 173
# igbo: 142
# hausa: 38
print(df[mask][20:40])

/tmp/ipykernel_2037/4115833663.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df["label"].eq("hausa") & df["text"].str.contains(pattern, case=False, regex=True)


                                                    text  label   format  \
26603  Game da faɗa kuma, ta ce a iya saninta bai taɓ...  hausa  article   
26604  Sai kuma a 2004, shekara ɗaya bayan Amurka ta ...  hausa  article   
26887  Shugaban Najeriya Bola Tinubu ya sanya hannu k...  hausa  article   
26966  Duk da haka, masana kimiyya da yawa sun yi ima...  hausa  article   
27347  Har yanzu ba a san inda 'ya'yansa shida suke b...  hausa  article   
27773  Haka shi ma Deji Adeyanju ɗaya daga cikin lauy...  hausa  article   
27797  Mun samu wuri a wani ɗakin karatu inda muka yi...  hausa  article   
28152  Sai dai Umm Hudaifa ba ta wurin - tana zaune a...  hausa  article   
28318  Bayan lamarin tsaro ya taɓarɓare a arewa maso ...  hausa  article   
28807  Wani kwararren masanin IT a sararin samaniyar ...  hausa  article   
29279  Wata majiya a ɓangaren tsaro a Iraƙi ta faɗa m...  hausa  article   
29475  Tuni aka san irin rashin imanin ƙungiyoyin da ...  hausa  article   
29550  Albar

In [9]:
for language in df["label"].unique():
    print(f"{language}: {len(df[df["label"]==language])}")

yoruba: 22610
igbo: 21126
hausa: 9493
english: 9235
pidgin: 14577


In [8]:
drop_list = ["To view this content choose", "I write to offer my warmest congratulations on your election", "We will continue to hunt for", "The pastor intervened in the", "Reports say the couple was having", "We were told that the priest", "Izunna Okafor is a lover of the Igbo language", "The government of Canada is not considering", "Osun State Police Spokesperson,","body has been deposited", "Reports say the man stabbed the priest", "The Dugbe Division Police rushed to the scene", "Her husband stabbed her several times before", "But the claimants say the tech firm recorded", "The 52-year-old pastor", "The Osun State Police Command has", "We praise God in this war for Prophet Mohammed", "She scored 18 goals"]
drop_index = [4181, 5869, 5302, 10144, 10420, 10996, 42656, 44365, 46101, 47225, 47792, 52072, 52348, 52542, 52688]
print(len(df))

77041


In [10]:
# Step 1: drop rows by index
df = df.drop(index=drop_index, errors="ignore")

# Step 2: drop rows containing any unwanted phrase in 'text'
pattern = "|".join(map(repr, drop_list))  # ensures proper escaping
pattern = "|".join(map(repr, drop_list)).replace("'", "")  # cleaner regex without quotes

mask = df["text"].str.contains(pattern, case=False, regex=True, na=False)
df = df[~mask]

# Reset index if needed
df = df.reset_index(drop=True)

print(f"Remaining rows: {len(df)}")

Remaining rows: 76976


In [11]:
for language in df["label"].unique():
    print(f"{language}: {len(df[df["label"]==language])}")

yoruba: 22579
igbo: 21092
hausa: 9493
english: 9235
pidgin: 14577


In [16]:
df["text"].duplicated().sum()

504

In [17]:
df = df.drop_duplicates(subset = ["text"])
df["text"].duplicated().sum()

0

In [19]:
len(df)

76472

In [18]:
train_dataset, eval_dataset = train_test_split(df, test_size = 0.2, random_state= 42)

In [20]:
import json

df.to_json("./datasets/language_detection/cleaned_dataset.jsonl", 
           orient="records",
           lines = True,
           force_ascii = False)
train_dataset.to_json("./datasets/language_detection/train_dataset.jsonl", 
           orient="records",
           lines = True,
           force_ascii = False)
eval_dataset.to_json("./datasets/language_detection/eval_dataset.jsonl", 
           orient="records",
           lines = True,
           force_ascii = False)

In [23]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="./datasets/language_detection/eval_dataset.jsonl")
print(dataset)

Generating train split: 15295 examples [00:00, 448802.85 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'format', 'source', 'citation'],
        num_rows: 15295
    })
})


In [24]:
LANGUAGES = ["english", "pidgin", "yoruba", "igbo", "hausa"]
label_mapping = {i: lang for i, lang in enumerate(LANGUAGES)} 
print(label_mapping)

{0: 'english', 1: 'pidgin', 2: 'yoruba', 3: 'igbo', 4: 'hausa'}


In [26]:
id2label = {v: k for k, v in label_mapping.items()}
print(id2label)

{'english': 0, 'pidgin': 1, 'yoruba': 2, 'igbo': 3, 'hausa': 4}


In [ ]:
# df[df["text"].str.contains("To view this content choose")] #16
# df[df["text"].str.contains("But Covington's fortunes changed when Usman")] #2
# df[df["text"].str.contains("I write to offer my warmest congratulations on your election")] #2
# df[df["text"].str.contains("We were told that the priest")] #2
# df[df["text"].str.contains("Izunna Okafor is a lover of the Igbo language.")] #2